In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency, normaltest

# I. Wybór tematu i zbioru danych

Projekt przygotowano z wykorzystaniem języka Python wraz z Anaconda oraz zbioru danych Heart Disease UCI.

Wybrany zbiór danych zawiera  składa się z następujących atrybutów:

age: wiek  
sex: płeć  
chest pain type (4 values): typ bólu w klatce piersiowej (0-bezobjawowy, 1-nietypowy ból serca, 2-ból nie związany z sercem, 3-typowy ból serca)  
resting blood pressure: spoczynkowe ciśnienie krwi  
serum cholestoral in mg/dl: poziom cholesterolu  
fasting blood sugar > 120 mg/dl: poziom cukru we krwi na czczo  
resting electrocardiographic results (values 0,1,2) : spoczynkowe EKG (0-prawdopodobny lub określony przerost lewej komory serca, 1-wynik w normie, 2-nieprawidłowość odcninka ST (odwrócenie załamka T i/lub uniesienie lub obniżenie odcinka ST > 0,05 mV)  
maximum heart rate achieved : maksymalna ilość uderzeń serca na minutę  
exercise induced angina : ból w klatce piersiowej wywołany ćwiczeniami  
oldpeak = ST depression induced by exercise relative to rest : obniżenie odcinka ST wywołane ćwiczeniami  
the slope of the peak exercise ST segment : nachylenie szczytowego odcinka ST ćwiczenia (0-opadające, 1- płaskie, 2- rosnące)  
number of major vessels (0-3) colored by flourosopy : liczba głównych naczyń krwionośnych oznaczonych podczas fluoroskopii  
thal: 1 = fixed defect; 2 = normal; 3 = reversable defect : wyniki badania z wykorzystaniem radioaktywnego kontrastu (1-wada nieodwracalna, 2-brak wad, 3-wada odwracalna)  
target : 0-stwierdzona choroba serca, 1- brak chorób serca  

Cel analizy Wybrano trzy hipotezy, które zostaną sprawdzone w poniższej pracy. Każda z hipotez została oparta o inną zmienną zależną.  
  
Hipoteza 1: Osoby w wieku poniżej 40 lat mające spoczynkowe ciśnienie krwi >=140 i odczuwający ból serca w klatce piersiowej podczas wysiłku chorują na serce.  

Hipoteza 2: Osoby nie chorujące na cukrzycę, nie mające podwyższonego ciśnieni krwi i cholesterolu odczuwają ból serca w klatce piersiowej podczas wysiłku.  
  
Hipoteza 3: Osoby bez stwierdzonej choroby serca, mające wysoki cholesterol i spoczynkowe ciśnienie krwi <140  chorują na cukrzycę.  

# II. Czyszczenie i analiza danych



In [ ]:
#Wczytanie bazy danych do lokalnej zmiennej
data = pd.read_csv("/kaggle/input/heart-disease-uci/heart.csv")
data['bps_high'] = np.where(data['trestbps']> 139, 'Yes', 'No')
data['age_low'] = np.where(data['age']< 40, 'Yes', 'No')
data['chol_high'] = np.where(data['chol']> 250, 'Yes', 'No')
data['sex'].replace(0, 'Female',inplace=True)
data['sex'].replace(1, 'Male',inplace=True)
data['fbs'].replace(0, 'No',inplace=True)
data['fbs'].replace(1, 'Yes',inplace=True)
data['exang'].replace(0, 'No',inplace=True)
data['exang'].replace(1, 'Yes',inplace=True)
data['target'].replace(0, 'Yes',inplace=True)
data['target'].replace(1, 'No',inplace=True)
data.head()




Dane wejściowe zostały pomyślnie wczytane, następnie sprawdzone zostaną pod kątem poprawności (pustych wierszy itp.)

In [ ]:
data.isna().sum()

Wybrany zbiór danych jest kompletny, dane są prawidłowo sformatowane, bez pustych luk.

1. Opracuj podstawowe statystyki dla każdej zmiennej ilościowej.
Dla wybranego zbioru zmienne ilościowe obejmują kolumny: age, trestbps, chol, thalach. I te kolumny zostaną poddane analizie statystycznej

In [ ]:
quant_col = ['age', 'trestbps', 'chol','thalach']
quant_stats = data[quant_col].agg(["count","mean","median","min", "max", "std", "var",])
quant_stats = quant_stats.append(data[quant_col].mode().rename(index={0:"mode"}).iloc[0])
quant_stats

2. Opracuj tabele liczności dla każdej zmiennej jakościowej z hipotez
Do zbadania hipotez wykorzystane zostaną zmienne jakościowe z kolumn: 'sex', 'cp', 'fbs', 'exang'. Poniżej przedstawione zostaną tabele liczności dla wartości tych zmiennych w formie histogramów

In [ ]:
quali_cols = ['sex','fbs', 'exang', 'target', 'bps_high', 'age_low', 'chol_high']
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i])
    print(f"""Dla kolumny {col} tabela liczności wygląda następująco:\n\n\n{pd.DataFrame(data[col].value_counts())}\n""")

Na powyższych wykresach można zauważyć, że rozkład prawdopodobieństwa dla płci nie jest równy, przeważają mężczyżni w stosunku 2:1, osoby chorujące na cukrzycę są w mniejszości (1:5), osoby odczuwające ból w klatce piersiowej to około połowa badanych. Natomiast rozkład osób chorych i zdrowych jest zbliżony.

In [ ]:
sns.histplot(x=data["sex"],hue=data["target"], multiple="stack")
data[["target", "sex"]].groupby(["sex","target"]).size()

Jak można zauważyć z powyższego histogramu, ponad połowa przebadanych mężczyzn choruje na serce. Natomiast wśród kobiet, tylko 1/5 przebadanych dotknięta jest chorobą. Na podstawie powyższych wykresów można wyciągnąć wczesny wniosek, że kobiety rzadziej niż mężczyźni mają problemy z sercem.

W celu zbadania hipotezy nr 1 utworzono tabele wielodzielcze względem kolumny 'target'.

In [ ]:
for col in quali_cols:
    if col != "target":
        print(pd.crosstab(data[col], data["target"], normalize="index"))
        print("\n\n")

Analizując wyniki tabel wielodzielczych dla hipotezy nr 1 możemy zauważyć to co przy poprzednim histogramie, czyli że występuje związek pomiędzy płcią a zdiagnozowaną chorobą serca, a także między wystąpieniem bólu w klatce piersiowej podczas ćwiczeń a zdiagnozowaną chorobą serca. W tym drugim przypadku wyniki nie są zasakujące, osoby odczuwające ból w klatce piersiowej podczas ćwiczeń było o około 76% częściej diagnozowane jako chore na serce. Natomiast tylko 23 % osób nie odczuwających bólu zostało zdiagnozowanych pozytywnie.

In [ ]:
fig, axes = plt.subplots(len(quali_cols), 1, figsize=(10,30))
for i, col in enumerate(quali_cols):
        axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
        sns.histplot(data = data, x=data[col], hue="target", ax=axes[i], multiple="dodge")

W celu zbadania hipotezy nr 2 utworzono tabele wielodzielcze względem kolumny 'exang'.

In [ ]:
for col in quali_cols:
    print(pd.crosstab(data[col], data["exang"], normalize="index"))
    print("\n\n")

Poza omówionym wcześniej związkiem bólu w klatce podczas wysiłku z chorobą serca nie uwidoczniły się wyraźne zależności

In [ ]:
fig, axes = plt.subplots(len(quali_cols[:-1]), 1, figsize=(10,30))
for i, col in enumerate(quali_cols[:-1]):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    sns.histplot(data = data, x=data[col], hue="exang", ax=axes[i], multiple="dodge")

W celu zbadania hipotezy nr 3 utworzono tabele wielodzielcze względem kolumny 'fbs'.

In [ ]:
for col in quali_cols:
    print(pd.crosstab(data[col], data["fbs"], normalize="index"))
    print("\n\n")

In [ ]:
fig, axes = plt.subplots(len(quali_cols[:-1]), 1, figsize=(10,30))
for i, col in enumerate(quali_cols[:-1]):
    axes[i].set_title(f"Wykres histogramu skategoryzowanego {col}")
    sns.histplot(data = data, x=data[col], hue="fbs", ax=axes[i], multiple="dodge")

Nie widać wyraźnych związków między wystąpieniem cukrzycy a pozostałymi zmiennymi.

3. Wykonanie macierzy korelacji

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data.corr(), annot=True, ax=ax)
data.corr()

age: istnieje wiele słabych korelacji wieku pacjentów ze zmiennymi trestbps(ciśnienie krwi), chol(cholesterol), thalach(maksymalne tętno), oldpeak(spadek odcinka ST podczas wysiłku w porównaniu do spoczynku), ca(ilość oznaczonych naczyń podczas fluoroskopii). Jest to prawdopodobne, że wiek wpływa negatywnie na parametry zdrowotne badanych osób.
cp: ból w klatce piersiowej (również jego brak) wykazuję największą korelację z thalach(maksymalne tętno).
trestbps: spoczynkowe ciśnienie wykazuję największą korelację z wiekiem pacjenta, jest to prawdopodobne, ponieważ istenieją badania wykazujące wzrost ciśnienia krwii wraz z wiekiem.
chol: występuje słaba korelacja dodatnia pomiędzy poziomem cholesterolu a wiekiem pacjenta
restecg: nie występują wyraźne korelacje z innymi zmiennymi
thalach: występuje kilka słabych korelacji ze zmiennymi age, cp, oldpeak, slope, ca.
oldpeak: występują dwie wyraźne korelacje ze zmiennymi thalach i slope
ca: występują słabe korelację ze zmiennymi age, thalach i oldpeak
thal: występuje słaba korelacja z oldpeak


4. Test niezależności przy użyciu testu chi^2 przy założeniu poziomu istotności alfa = 0.05, zakładając za hipotezę zerową, że zmienne są niezależne.

Sprawdzamy hipotezę nr 1 (zmienna target)

In [ ]:
cols_to_drop = ["target"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["target"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

Zmienne sex, cp, restecg, exang, oldpeak, slope, ca,thal, bps_high, chol_high są zależne ze zmienną target.

Sprawdzamy hipotezę nr 2 (zmienna exang)

In [ ]:
cols_to_drop = ["exang"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["exang"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

Zmienne sex, cp, thalach, oldpeak, slope, ca, thal, target są zależne ze zmienną exang.

Sprawdzamy hipotezę nr 3 (zmienna fbs)

In [ ]:
cols_to_drop = ["fbs"]
for col in data.columns:
    if col not in cols_to_drop:
        contigency = pd.crosstab(data[col], data["fbs"])
        chi,p_value,degrees_of_freedom , expected_freq = chi2_contingency(contigency)
        print(f"Dla kolumny {col} wartość p testu niezależności wynosi {p_value}")
        if p_value <= 0.05:
            print(f"Wartość p jest mniejsza dla założonego poziomu istotności co pozwala na odrzucenie hipotezy zerowej - zmienne są zależne\n")
        else:
            print(f"Wartość p jest większa dla założonego poziomu istotności co potwierdza hipotezę zerową - zmienne są niezależne\n")

Żadna ze zmiennych nie jest zależna ze zmienną fbs. Potwierdza to wcześniejsze wyniki, co może oznaczać że hipoteza nr 3 nie będzie możliwa do potwierdzenia.

5. Wykonaj wykresy ramka-wąsy dla wszystkich zmiennych ilościowych z hipotez. Wybierz dwie pary zmiennych (ilościowa-jakościowa) i wykonaj wykresy

In [ ]:
fig, axes = plt.subplots(len(quant_col),1,  figsize=(10,30))
for i, col in enumerate(quant_col):
    axes[i].set_title(f"Wykres pudełkowy kolumny {col}")
    sns.boxplot(data = data, x=data[col], ax=axes[i], orient = "h")

Wykresy pudełkowe skategoryzowane wykonano dla par zmiennych jakościowa z ilościową: (target, age), (exang, trestbps), (fbs, chol). Nie zdecydowano się na usunięcie wartości skrajnych ponieważ są ważne z punktu widzenia diagnostyki pacjentów i mogą wpłynąć na ostateczne wnioski.

In [ ]:
sns.boxplot(data = data, x="target", y = "age", orient="v")

In [ ]:
sns.boxplot(data = data, x="exang", y = "trestbps", orient="v")

In [ ]:
sns.boxplot(data = data, x="fbs", y = "chol", orient="v")

Wykresy skategoryzowane wykazały ponownie związek wieku pacjenta z diagnozą choroby serca.

6. Wykonaj test normalny dla zmiennych - oznacz wartości odstające

W pierwszym kroku zbadano zmienne ilościowe w celu określenia przydatności danych zmiennych. 

In [ ]:
fig, axes = plt.subplots(len(quant_col), 1, figsize=(10,30))
for i, col in enumerate(quant_col):
    axes[i].set_title(f"Wykres rozkładu liczności dla {col}")
    sns.histplot(data[col], ax=axes[i], kde=True)

In [ ]:
for i, col in enumerate(quant_col):
    print(f"Wartość p dla testu normalnego dla kolumny {col} wynosi {normaltest(data[col].values).pvalue}\n")

Jak widać wszystkie wykresy zmiennych ukazują że są one przybliżeniami rozkładu normalnego.
Z tego powodu wykonano wykresy rozrzutu dla wszystkich zmiennyh w celu znalezienia zależności między nimi.

In [ ]:
disp = sns.PairGrid(data, hue="target")
disp.map_diag(sns.histplot)
disp.map_offdiag(sns.scatterplot)
disp.add_legend()

# III Indukcja drzew decyzyjnych

1. Utwórz drzewo klasyfikacyjne dla zmiennej jakościowej. Jeśli zmienna zależna jest ilościowa, utwórz drzewo regresyjne.

W celu weryfikacji hipotezy nr 1 wybrano następujące zmienne:

age
trestbps
exang

In [ ]:

# improtowanie potrzebnych bibliotek
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, precision_recall_fscore_support
pd.options.mode.chained_assignment = None

# oddzielenie potrzebnych danych
hipo_1_data = data[["age", "trestbps", "exang"]]
hipo_1_y = data[["target"]]

decode = {
    "Yes" : 1,
    "No" : 0
}

hipo_1_data.loc[:, "exang"] = hipo_1_data.exang.map(decode).values
hipo_1_y.loc[:, "target"] = hipo_1_y.target.map(decode).values

# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_1_data, hipo_1_y, train_size=0.7, random_state=1)
X_train

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=15)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

2. Określ ważność predyktorów z użyciem wykresu.

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_1_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_1_data.shape[1]), hipo_1_data.columns)
plt.ylim([-1, hipo_1_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns, proportion=True, class_names=["No disease", "Heart Disease"])
plt.show()

In [ ]:
plot_confusion_matrix(tree_model, X_test, y_test, display_labels=["No disease", "Heart disease"])
precission, recall, f_score, support = precision_recall_fscore_support(y_test, preds)
print(f"Dokładność (precission) modelu podczas określania czy dana osoba choruje na serce {precission[1]:.2f}, natomiast rozpoznanie ilość (recall) dla tej klasy wynosi {recall[1]:.2f}")
print(f"Oznacza to, że {100 * precission[1]:.2f}% osób, które zostały sklasyfikowane chore zostało poprawnie sklasyfikowanych, a {100 * recall[1]:.2f}% wszystkich osób chorych zostało poprawnie sklasyfikowanych.")

Na podstawie analizy drzewa decyzyjnego można zauważyć, że wystąpienie bólu w klatce piersiowej podczas ćwiczeń wyraźnie wpływa na zwiększenie prawdopodobieństwa choroby serca. Można też zauważyć związki z wyższym wiekiem i wysokim ciśnieniem krwi a zwiększonym prawdopodobieństwem choroby.

W celu weryfikacji hipotezy nr 2 wybrano następujące zmienne:

fbs
trestbps
chol

In [ ]:
# oddzielenie potrzebnych danych
hipo_2_data = data[["fbs", "trestbps", "chol"]]
hipo_2_y = data[["exang"]]

hipo_2_y.loc[:, "exang"] = hipo_2_y.exang.map(decode).values
hipo_2_data.loc[:, "fbs"] = hipo_2_data.fbs.map(decode).values

# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_2_data, hipo_2_y, train_size=0.7, random_state=1)

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=10)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_2_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_2_data.shape[1]), hipo_2_data.columns)
plt.ylim([-1, hipo_2_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns, class_names=["No chest pain","Pain during exercise"])
plt.show()

In [ ]:
plot_confusion_matrix(tree_model, X_test, y_test, display_labels=["No chest pain","Pain during exercise"])
precission, recall, f_score, support = precision_recall_fscore_support(y_test, preds)
print(f"Dokładność (precission) modelu podczas określania czy dana osoba cierpi na ból w klatce piersiowej podczas ćwiczeń {precission[1]:.2f}, natomiast rozpoznanie ilość (recall) dla tej klasy wynosi {recall[1]:.2f}")
print(f"Oznacza to, że {100 * precission[1]:.2f}% osób, które odczuwały ból zostało poprawnie sklasyfikowanych, a {100 * recall[1]:.2f}% wszystkich osób odczuwających ból zostało poprawnie sklasyfikowanych.")

Z analizy drzewa decyzyjnego nie można potwierdzić hipotezy nr 2. Wnioski są niejednoznaczne, nie został ujawniony widoczyny związek z występowaniem bólu w klatce piersiowej podczas ćwiczeń.

W celu weryfikacji hipotezy nr 3 wybrano następujące zmienne:

target
trestbps
chol

In [ ]:
# oddzielenie potrzebnych danych
hipo_3_data = data[["target", "trestbps", "chol"]]
hipo_3_y = data[["fbs"]]

hipo_3_y.loc[:, "fbs"] = hipo_3_y.fbs.map(decode).values
hipo_3_data.loc[:, "target"] = hipo_3_data.target.map(decode).values

# rozdzielenie danych na sety treningowe i testowe
X_train, X_test, y_train, y_test = train_test_split(hipo_3_data, hipo_3_y, train_size=0.5, random_state=1)

# ładowanie modelu
tree_model = DecisionTreeClassifier(min_samples_leaf=5)

# trenowanie modelu
tree_model.fit(X_train, y_train)

# sprawdzanie dokłądności modelu na danych testowych

preds = tree_model.predict(X_test)
acc = accuracy_score(y_test, preds)
print(f"Dokładność modelu wynosi {100*acc:.2f}%")

In [ ]:
var_importances = tree_model.feature_importances_
std = np.std(var_importances,axis=0)
indices = np.argsort(var_importances)
plt.figure()
plt.title("Ważność predyktorów")
plt.barh(range(hipo_3_data.shape[1]), var_importances[indices],
       color="r", xerr=std, align="center")
plt.yticks(range(hipo_3_data.shape[1]), hipo_3_data.columns)
plt.ylim([-1, hipo_3_data.shape[1]])
plt.grid(b=True)
plt.xlim(0, 1)
plt.show()

In [ ]:
plt.figure(figsize=(25,20))
plot_tree(tree_model, filled=True, rounded=True, feature_names=X_train.columns, class_names=["No diabetes","Diabetes"])
plt.show()

In [ ]:
plot_confusion_matrix(tree_model, X_test, y_test, display_labels=["No diabetes","Diabetes"])
precission, recall, f_score, support = precision_recall_fscore_support(y_test, preds)
print(f"Dokładność (precission) modelu podczas określania czy dana osoba choruję na cukrzycę {precission[1]:.2f}, natomiast rozpoznanie ilość (recall) dla tej klasy wynosi {recall[1]:.2f}")
print(f"Oznacza to, że {100 * precission[1]:.2f}% osób, które chorowały na cukrzycę zostało poprawnie sklasyfikowanych, a {100 * recall[1]:.2f}% wszystkich osób chorujących na cukrzycę zostało poprawnie sklasyfikowanych.")

Z analizy drzewa decyzyjnego nie wynikają żadne zależności między wystąpieniem cukrzycy a pozostałymi zmiennymi badanymi. 

# IV. Analiza skupień

1. Wybierz zmienne do analizy - uzasadnij.

Do wykonania analizy skupień wybrano zmienne ilościowe ponieważa analiza skupień nie działa zbyt dobrze dla zmiennych binarnych:

age
trestbps
thalach
chol

Celem analizy jest sprawdzenie, czy jesteśmy w stanie wyznaczyć jakieś konkretne grupy osób z konkretnymi schorzeniami, które mogą wpływać na prawdopodobieństwo choroby serca.

In [ ]:
# improtowanie potrzebnych bibliotek
from sklearn.cluster import KMeans
pd.options.mode.chained_assignment = None
from sklearn.decomposition import PCA

# oddzielenie potrzebnych danych
cluster_data = data[["age", "trestbps", "thalach", "chol"]]

sse = []

# ładowanie modelu
for k in range(1,11):
    cluster_model = KMeans(n_clusters=k)
    cluster_model.fit(cluster_data)
    sse.append(cluster_model.inertia_)
    
# sprawdzanie jaką wartość k wybrać - metoda łokcia

plt.style.use("fivethirtyeight")
plt.plot(range(1, 11), sse)
plt.xticks(range(1, 11))
plt.xlabel("Wartość k")
plt.ylabel("SSE")
plt.show()


Z powyższego wykresu wynika, że liczba klastrów do analizy powinna wynosić 2. Zdecydowano się zweryfikować to poprzez użycie silhouette coefficient score w celu automatycznego doboru ilości klastrów.

Silhouette Coefficient jest obliczany przy użyciu średniej odległości wewnątrz klastra (a) i średniej odległości do najbliższego klastra (b) dla każdej próbki. Współczynnik sylwetki dla próbki to (b - a) / max (a, b). Aby wyjaśnić, b to odległość między próbką a najbliższą gromadą, której próbka nie jest częścią. Zwróć uwagę, że silhouette coefficient jest definiowany tylko wtedy, gdy liczba etykiet wynosi 2 <= n_labels <= n_samples - 1.

In [ ]:
from sklearn.metrics import silhouette_score

sil_score_max = -1
best_n_clusters = 0

for k in range(2,11):
  cluster_model = KMeans(n_clusters = k)
  labels = cluster_model.fit_predict(cluster_data)
  sil_score = silhouette_score(cluster_data, labels)
  print(f"Średnia wartość silhouette score dla {k} klastrów wynosi {sil_score}")
  if sil_score > sil_score_max:
    sil_score_max = sil_score
    best_n_clusters = k
    
print(f"Najlepsza ilość klastrów to: {best_n_clusters}")

Automatyczny dobór ilości klastrów także wybrał wartość: 2

In [ ]:
cluster_model = KMeans(n_clusters = best_n_clusters)
# trenowanie modelu
result = cluster_model.fit_predict(cluster_data)

labels = cluster_model.labels_

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra 1:

In [ ]:
results_0.describe()

Analiza klastra 2:

In [ ]:
results_1.describe()

Można zauważyć że rozkład zmiennych jest jednolity. Największe odchylenia można zauważyć dla zmiennej chol.

2. Analiza skupień metodą EM.

In [ ]:
# załadowanie modułu do analizy metodą EM
from sklearn.mixture import GaussianMixture

# wczytanie modelu z dwoma klastrami
em_model = GaussianMixture(n_components=2, random_state=0)

labels = em_model.fit_predict(cluster_data)

result_data = cluster_data.copy()
result_data["labels"] = labels

results_0 = cluster_data[result_data.labels == 0]
results_1 = cluster_data[result_data.labels == 1]

Analiza klastra 1:

In [ ]:
results_0.describe()

Analiza klastra 2:

In [ ]:
results_1.describe()

Porównując wyniki pomiędzy analizą skupień metodą K-najbliższych można zauważyć że metod EM bardziej różnorodnie podzieliła dane, aczkolwiek różnice nie są duże. Większe odchylenia można zauważyć dla zmiennych trestbps, thalach i chol.

In [ ]:
# rysowanie wykresu dla każdej zmiennej 
import scipy.stats as stats
import math

for col in results_1:
    mu_0 = results_0[col].mean()
    variance_0 = results_0[col].var()
    mu_1 = results_1[col].mean()
    variance_1 = results_1[col].var()
    sigma_0 = math.sqrt(variance_0)
    sigma_1 = math.sqrt(variance_1)
    x_0 = np.linspace(mu_0 - 3*sigma_0, mu_0 + 3*sigma_0, 100)
    x_1 = np.linspace(mu_1 - 3*sigma_1, mu_1 + 3*sigma_1, 100)
    plt.figure(figsize=(8,5))
    plt.title(f"Wykres dystrybucji zmiennych w klastrach dla zmiennej {col}")
    plt.plot(x_0, stats.norm.pdf(x_0, mu_0, sigma_0))
    plt.plot(x_1, stats.norm.pdf(x_1, mu_1, sigma_1))
    plt.show()

Z powyższej analizy można zauważyć związku pomiędzy spoczynkowym ciśnieniem krwi a cholesterolem i maksymalnym tętnem.

# V Algorytm Data Mining

Zdecydowano się na wybór algrytmu SVM (Support Vector Machines) jako algorytmu Data Mining. Do analizy użyto wszystkie zmienne za wyjątkiem age_low, bps_high, chol_high, które zostały dodane na początku analizy. Ze względu na pozostawienie wartości skrajnych wykonywane jest wyłącznie skalowanie.

In [ ]:
data_2 = data
data_2 = data_2.drop(['age_low', 'bps_high', 'chol_high'], axis=1)
data_2['sex'].replace('Female', 0,inplace=True)
data_2['sex'].replace('Male', 1,inplace=True)
data_2['fbs'].replace('No', 0,inplace=True)
data_2['fbs'].replace('Yes', 1,inplace=True)
data_2['exang'].replace('No', 0,inplace=True)
data_2['exang'].replace('Yes', 1,inplace=True)
data_2['target'].replace('Yes', 1,inplace=True)
data_2['target'].replace('No', 0,inplace=True)

Y = data_2.target
x = data_2.drop('target', axis = 1)
columns = x.columns
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(x)
data_x = pd.DataFrame(X, columns = columns)


Następuje podzielenie na zbioru uczące i testowe.

In [ ]:
#Splitting the data into training and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_x, Y, test_size = 0.15, random_state = 45)

W celu usunięcia różnic wyważenia w danych uczących użyto SMOTE.

In [ ]:
from imblearn.over_sampling import SMOTE
smt = SMOTE()
x_train, y_train = smt.fit_resample(x_train, y_train)
np.bincount(y_train)

Następnie model dopasowano z wykorzystaniem regresji logistycznej.

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(x_train, y_train)
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))

Dla porównania dopasowano model na podstawie SVM (Support Vector Machine):

In [ ]:
from sklearn.svm import SVC
classifier_rbf = SVC(kernel = 'rbf')
classifier_rbf.fit(x_train, y_train)
y_pred = classifier_rbf.predict(x_test)
print('Accuracy of SVC (RBF) classifier on test set: {:.2f}'.format(classifier_rbf.score(x_test, y_test)))

Dla porównania dopasowano model metodą Random Forest z 255 drzewami:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=255, bootstrap = True, max_features = 'sqrt')
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print('Accuracy of Random Forest on test set: {:.2f}'.format(model.score(x_test, y_test)))

Można zauważyć, że największą dokładność uzyskały modele LogisticRegression i SVM, nastomiast trochę gorzej dopasowany został model Random Forest.

# VI Podsumowanie

Z całej powyższej analizy najbardziej prawdopodobna wydaję się hipoteza nr 1. Niestety z powodu małej ilości badanych osób w wieku poniżej 40 lat nie jesteśmy w stanie stwierdzić czy młody wiek ma mocny wpływ na chorobę, natomiast jesteśmy wstanie stwierdzić zależność, że im starsza osoby i im gorsze jej parametry zdrowotne tym większe prawdopodobieństwo choroby. Takie wnioski nie są zaskakujące, przez to ich potwierdzenie w wyniku analizy danych może wpłynąć na zwiększenie jej wiarygodności. Ból w klatce piersiowej i wysokie ciśnienie spoczynkowe oznaczają, że choroba serca jest bardziej prawdopodobna i osoba z takimi objawami powinna udać się do lekarza. Hipoteza nr 2 nie znajduję jednoznacznego potwierdzenia w powyższej analizie. Według analizy cukrzyca nie ma związku z bólem w klatce piersiowej, natomiast cholesterol i ciśnienie krwii nie wykazały jednoznaczego związku. Natomiast te objawy mogą spowodować chorobę serca w przyszłości ze względu na to, że ich wystąpienie obciąża organizm. Z tego powodu ból w klatce piersioewj podczas ćwiczeń może wystąpić po czasie ale nie musi. Hipoteza nr 3 nie znajduję żadnych potwierdzeń w przeprowadzonej analizie. Nie wykazano związku występowania cukrzycy z innymi zmiennymi. Może to być spowodowane małą ilością osób chorych na cukrzycę w badanej grupie. Podsumowując analiza danych nie wykazała, żadnych niespotykanych dotąd zależności pomiędzy parametrami zdrowotnymi a chorobą serca, złe parametry zdrowotne zwiększają prawdopodobieństwo chorób, w tym choroby serca.

Przygotował: Miłosz Ryba